# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I
output_data_file = pd.read_csv("../WeatherPy/output_data/cities.csv")
output_data_file.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Marystown,100,CA,1607561475,80,47.17,-55.15,38.64,21.23
1,Punta Arenas,100,CL,1607561414,76,-53.15,-70.92,50.00,2.24
2,Barrow,90,US,1607561476,70,71.29,-156.79,-5.80,14.99
3,Albany,86,US,1607561476,73,42.60,-73.97,34.00,3.29
4,Nālūt,13,LY,1607561476,60,30.33,10.85,49.50,4.85


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = output_data_file[["Lat", "Lng"]]

# Store humidity in humidity
humidity = output_data_file["Humidity"]

In [4]:
# Plot Heatmap
m = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
m.add_layer(heatmap_layer)

#Display figure (m)
m

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
output_data_file

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Marystown,100,CA,1607561475,80,47.17,-55.15,38.64,21.23
1,Punta Arenas,100,CL,1607561414,76,-53.15,-70.92,50.00,2.24
2,Barrow,90,US,1607561476,70,71.29,-156.79,-5.80,14.99
3,Albany,86,US,1607561476,73,42.60,-73.97,34.00,3.29
4,Nālūt,13,LY,1607561476,60,30.33,10.85,49.50,4.85
...,...,...,...,...,...,...,...,...,...
560,Salalah,0,OM,1607561543,25,17.02,54.09,73.40,6.93
561,Glens Falls,90,US,1607561543,92,43.31,-73.64,32.00,8.05
562,Gushikawa,40,JP,1607561543,94,26.35,127.87,69.80,5.82
563,Road Town,75,VG,1607561535,88,18.42,-64.62,78.01,2.24


In [6]:
# Narrow down the cities with wind speed less than 20 mph, max_temp between 20 and 30 with humidity of less than 80%
narrowed_city_df = output_data_file.loc[(output_data_file['Wind Speed'] < 20) & (output_data_file['Max Temp'] <= 30) & (output_data_file['Humidity'] <= 65)]
(output_data_file['Max Temp'] <= 30).dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
80,Ilulissat,100,GL,1607561484,41,69.22,-51.10,21.2,6.93
86,Tasiilaq,0,GL,1607561485,55,65.61,-37.64,8.6,3.67
190,Udachny,40,RU,1607561498,65,66.42,112.40,-40.0,4.47
380,Petropavlovsk-Kamchatskiy,0,RU,1607561520,57,53.05,158.65,17.6,8.95


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrowed_city_df.loc[:,["City", "Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the dataframe
hotel_df["Hotel"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel
80,Ilulissat,GL,69.22,-51.10,
86,Tasiilaq,GL,65.61,-37.64,
190,Udachny,RU,66.42,112.40,
380,Petropavlovsk-Kamchatskiy,RU,53.05,158.65,


In [8]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set up a parameters dictionary
params = {"type" : "hotel",
         "keyword" : "hotel",
         "radius" : 5000,
         "key" : g_key}

In [9]:
# Start for loop
for index, row in hotel_df.iterrows():
    # get city name, lat and lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response ['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    
    # if there is not hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result...skipping.")
        
        # Wait 1 second to make another api request to avoid SSL error
        time.sleep(1)

# Print end of search once searching is completed
print("----------End of Search----------")

Retrieving Results for Index 80: Ilulissat.
Closest hotel in Ilulissat is Hotel Arctic.
Retrieving Results for Index 86: Tasiilaq.
Closest hotel in Tasiilaq is The Red House.
Retrieving Results for Index 190: Udachny.
Closest hotel in Udachny is Gostinitsa Vilyuy.
Retrieving Results for Index 380: Petropavlovsk-Kamchatskiy.
Closest hotel in Petropavlovsk-Kamchatskiy is Avacha Hotel.
----------End of Search----------


In [10]:
hotel_df

,City,Country,Lat,Lng,Hotel,Hotel Name
80,Ilulissat,GL,69.22,-51.10,,Hotel Arctic
86,Tasiilaq,GL,65.61,-37.64,,The Red House
190,Udachny,RU,66.42,112.40,,Gostinitsa Vilyuy
380,Petropavlovsk-Kamchatskiy,RU,53.05,158.65,,Avacha Hotel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

#Add the layer to the map
m.add_layer(markers)

# Display figure
m

Figure(layout=FigureLayout(height='420px'))